In [72]:
import numpy as np
import pandas as pd


class MyLineReg():
    def __init__(self, n_iter, learning_rate, metric=None, reg=None, l1_coef=0, l2_coef=0):
        self.n_iter = n_iter
        self.learning_rate = self.learning_rate_type(learning_rate)
        self.weights = None
        self.metric = metric
        self.metrics()
        self.reg = reg
        self.l1_coef = l1_coef
        self.l2_coef = l2_coef
        

    def calculate_gradient(self, X, y, y_pred):
        if self.reg == 'l1':
            return (2/X.shape[0]) * np.dot((y_pred - y.values.ravel()), X) + self.l1_coef*np.sign(self.weights)
        if self.reg == 'l2':
            return (2/X.shape[0]) * np.dot((y_pred - y.values.ravel()), X) + self.l2_coef*2*(self.weights)
        if self.reg == 'elasticnet':
            assert self.l1_coef != 0 and self.l2_coef != 0 
            return (2/X.shape[0]) * np.dot((y_pred - y.values.ravel()), X) + self.l1_coef*np.sign(self.weights) + self.l2_coef*2*(self.weights)
        else:
            return (2/X.shape[0]) * np.dot((y_pred - y.values.ravel()), X)

    def learning_rate_type(self, LR):
        if not isinstance(LR, float):
            return LR
        else:
            return lambda x: LR

    def metrics(self) -> None:
        if self.metric:
            if self.metric == 'mae':
                self.metric = ['mae', lambda y,
                               y_pred: np.mean(np.abs(y.values - y_pred))]

            if self.metric == 'mse':
                self.metric = ['mse', lambda y,
                               y_pred: np.mean((y.values - y_pred)**2)]

            if self.metric == 'rmse':
                self.metric = ['rmse', lambda y, y_pred: (
                    np.mean((y.values - y_pred)**2))**(0.5)]

            if self.metric == 'mape':
                self.metric = ['mape', lambda y, y_pred: 100 *
                               np.mean(np.abs((y.values - y_pred)/y))]

            if self.metric == 'r2':
                self.metric = ['r2', lambda y, y_pred: (1 -
                               (np.sum((y.values - y_pred)**2))/(np.sum((y.values - np.mean(y.values))**2)))]

    def __str__(self):
        return f"MyLineReg class: n_iter={self.n_iter}, learning_rate={self.learning_rate}"

    def __repr__(self):
        return f"MyLineReg class: n_iter={self.n_iter}, learning_rate={self.learning_rate}"

    def fit(self, samples: pd.DataFrame, y: pd.Series, verbose=False) -> None:

        X = samples.copy()
        X.insert(0, 'bias', pd.Series(1, index=range(X.shape[0])))

        self.weights = np.ones(X.shape[1])
        for iter in range(1, self.n_iter+1):
            y_pred = np.dot(X, self.weights)
            loss = np.mean((y_pred - y.values)**2)


            grad = self.calculate_gradient(X, y, y_pred)


            self.weights = self.weights - grad * self.learning_rate(iter)

            if verbose and (iter % verbose) == 0 and self.metric is not None:
                print(
                    f'iter = {iter+1} ||| Loss = {loss} ||| {self.metric[0]} = {self.metric[1](y, y_pred)}')
            elif verbose and (iter % verbose) == 0:
                print(f'iter = {iter+1} ||| Loss = {loss}')
            if self.metric:
                self.final_metric = self.metric[1](y, np.dot(X, self.weights))

    def predict(self, samples: pd.DataFrame) -> int:
        """
        Принимает на вход  матрицу фичей в виде датафрейма пандаса.
        Дополняет матрицу фичей единичным вектором (первый столбец).
        Возвращает вектор предсказаний.
        """

        X = samples.copy()
        X.insert(0, 'bias', pd.Series(1, index=range(X.shape[0])))

        return np.dot(X, self.weights)

    def get_coef(self) -> list():
        try:
            assert self.weights is not None
            return np.array(self.weights[1:])
        except:
            return 'fit before!'

    def get_best_score(self) -> int:
        return self.final_metric

In [73]:
X = pd.DataFrame(range(1000))
y = (pd.DataFrame(list(range(1000)))*150)

model = MyLineReg(50, lambda iter: 0.5 * (0.85 ** iter), 'r2', reg='l2', l2_coef=1)

In [74]:
# model.fit(X, y, 10)
# model.get_coef()

In [79]:
qw = model.learning_rate(0)

In [80]:
qw

0.5

In [45]:
q = 

In [47]:
for i in range(20):
    print(q(i))

0.5
0.425
0.36124999999999996
0.30706249999999996
0.26100312499999995
0.22185265624999997
0.18857475781249997
0.16028854414062496
0.13624526251953123
0.11580847314160153
0.0984372021703613
0.0836716218448071
0.07112087856808604
0.06045274678287313
0.05138483476544216
0.043677109550625835
0.037125543118031956
0.03155671165032716
0.026823204902778088
0.022799724167361375
